In [1]:
import sys
import ast

In [2]:
# !{sys.executable} -m pip install numpy==1.26.4
# !{sys.executable} -m pip install unsloth vllm==0.7.2
# !{sys.executable} -m pip install -U ipywidgets

In [3]:
# from unsloth import FastLanguageModel, is_bfloat16_supported
import torch
import re
from datasets import load_dataset

In [4]:
from unsloth import FastLanguageModel, is_bfloat16_supported

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Failed to patch Gemma3ForConditionalGeneration.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 04-20 22:20:19 __init__.py:190] Automatically detected platform cuda.


In [5]:
from transformers import (
    AutoTokenizer,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq,
    TrainingArguments,
    Trainer
)

In [6]:
SYSTEM_PROMPT = '''
You are playing the NY Times Connections game. Your task is to categorize 16 given words into exactly 4 groups of 4 words each, based on shared common themes.

Only output your final solution in the following format:
<answer>
[['WORD1', 'WORD2', 'WORD3', 'WORD4'],
 ['WORD5', 'WORD6', 'WORD7', 'WORD8'],
 ['WORD9', 'WORD10', 'WORD11', 'WORD12'],
 ['WORD13', 'WORD14', 'WORD15', 'WORD16']]
</answer>

Rules:
	• Each word must belong to one group only.
	• Groups must have a clear, shared theme (e.g., weather, NBA teams, keyboard keys, etc.).
	• Do not include any words not present in the input list.

Here is an example:

USER: [BUCKS, HAIL, JAZZ, SHIFT, LEVEL, MOM, SNOW, RACECAR, SLEET, TAB, KAYAK, RETURN, OPTION, NETS, RAIN, HEAT]

SOLUTION:
[['HAIL', 'RAIN', 'SLEET', 'SNOW'],
 ['BUCKS', 'HEAT', 'JAZZ', 'NETS'],
 ['OPTION', 'RETURN', 'SHIFT', 'TAB'],
 ['KAYAK', 'LEVEL', 'MOM', 'RACECAR']]

Explanation:
- WEATHER TERMS: 'HAIL', 'RAIN', 'SLEET', 'SNOW'
- NBA TEAMS: 'BUCKS', 'HEAT', 'JAZZ', 'NETS'
- KEYBOARD KEYS: 'OPTION', 'RETURN', 'SHIFT', 'TAB'
- PALINDROMES: 'KAYAK', 'LEVEL', 'MOM', 'RACECAR'
'''

In [7]:
def get_connections_questions(split="train"):
    data = load_dataset('csv', data_files='../data/final_transformed_connections.csv')[split]
    data = data.train_test_split(test_size=0.1, seed=3407)
    train_data = data['train']
    test_data  = data['test']
    # each x has 'questions' (the 16‐word list) and 'answer' (the reference)
    train_data = train_data.map(lambda x: {
        'prompt': [
            {'role': 'assistant', 'content': SYSTEM_PROMPT},
            {'role': 'user',      'content': x['questions']}
        ],
        'answer': x['answer']
    })
    test_data = test_data.map(lambda x: {
        'prompt': [
            {'role': 'assistant', 'content': SYSTEM_PROMPT},
            {'role': 'user',      'content': x['questions']}
        ],
        'answer': x['answer']
    })
    return train_data, test_data

train_ds, test_ds = get_connections_questions()

In [8]:
# ——————————————————————————————
# 2) Initialize model + LoRA adapter
# ——————————————————————————————

max_seq_length = 2048
lora_rank       = 8

# load base model
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name            = "unsloth/Llama-3.2-3B-Instruct",
    max_seq_length        = max_seq_length,
    load_in_4bit          = True,    # 4‑bit base + LoRA
    fast_inference        = True,
    max_lora_rank         = lora_rank,
    gpu_memory_utilization= 0.9,
)

# wrap with PEFT
model = FastLanguageModel.get_peft_model(
    model              = model,
    r                  = lora_rank,
    target_modules     = ["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"],
    lora_alpha         = lora_rank,
    random_state       = 3407,
)

# Make sure we also have a HF‐style tokenizer
hf_tokenizer = AutoTokenizer.from_pretrained("unsloth/Llama-3.2-3B-Instruct", use_fast=True)
hf_tokenizer.pad_token_id = hf_tokenizer.eos_token_id


==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.51.3. vLLM: 0.7.2.
   \\   /|    Tesla V100-PCIE-32GB. Num GPUs = 1. Max memory: 31.733 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 7.0. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Your GPU does not support prefix caching - will disable!
Unsloth: vLLM loading unsloth/llama-3.2-3b-instruct-unsloth-bnb-4bit with actual GPU utilization = 89.03%
Unsloth: Your GPU has CUDA compute capability 7.0 with VRAM = 31.73 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 2048. Num Sequences = 320.
Unsloth: vLLM's KV Cache can use up to 25.85 GB. Also swap space = 6 GB.
WARNING 04-20 22:20:27 config.py:2386] Casting torch.bfloat16 to torch.float16.
INFO 04-20 22:20:34 config.py:

[W420 22:20:36.102123370 CUDAAllocatorConfig.h:28] Warning: expandable_segments not supported on this platform (function operator())


INFO 04-20 22:20:38 weight_utils.py:252] Using model weights format ['*.safetensors']


model.safetensors:   0%|          | 0.00/2.35G [00:00<?, ?B/s]

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


...

INFO 04-20 22:24:45 model_runner.py:1562] Graph capturing finished in 31 secs, took 0.76 GiB
INFO 04-20 22:24:45 llm_engine.py:431] init engine (profile, create kv cache, warmup model) took 37.40 seconds



Unsloth 2025.3.19 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


tokenizer_config.json:   0%|          | 0.00/54.7k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

In [9]:
# ——————————————————————————————
# 3) Preprocess: pad labels to 1024
# ——————————————————————————————
def preprocess_sft(batch):
    inputs = [
        tokenizer.apply_chat_template(p, tokenize=False, add_generation_prompt=False)
        for p in batch["prompt"]
    ]
    model_inputs = hf_tokenizer(
        inputs,
        max_length = max_seq_length,
        padding    = "max_length",
        truncation = True,
    )
    labels = hf_tokenizer(
        batch["answer"],
        max_length = max_seq_length,    # ← pad answers out to 1024
        padding    = "max_length",
        truncation = True,
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

train_enc = train_ds.map(
    preprocess_sft, batched=True, remove_columns=train_ds.column_names
)
test_enc  = test_ds.map(
    preprocess_sft, batched=True, remove_columns=test_ds.column_names
)

data_collator = DataCollatorForSeq2Seq(
    tokenizer           = hf_tokenizer,
    model               = model.model,   # unwrapped base
    label_pad_token_id  = -100,          # ignore padded labels
)

Map:   0%|          | 0/580 [00:00<?, ? examples/s]

Map:   0%|          | 0/65 [00:00<?, ? examples/s]

In [11]:
# ——————————————————————————————
# 4) Seq2SeqTrainer with padded labels
# ——————————————————————————————
sft_args = TrainingArguments(
    output_dir                  = "sft_outputs",
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 16,
    num_train_epochs            = 3,
    learning_rate               = 5e-5,
    fp16                        = not is_bfloat16_supported(),
    bf16                        = is_bfloat16_supported(),
    logging_steps               = 10,
    save_steps                  = 100,
    save_total_limit            = 2,
)

sft_trainer = Trainer(
    model         = model,
    args          = sft_args,
    train_dataset = train_enc,
    eval_dataset  = test_enc,
    tokenizer     = hf_tokenizer,
    data_collator = DataCollatorForSeq2Seq(
        tokenizer          = hf_tokenizer,
        model              = model.model,
        label_pad_token_id = -100,
    ),
)

/tmp/SLURM_67392/ipykernel_3911059/2740054933.py:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  sft_trainer = Trainer(
Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [12]:
# run the SFT pass
sft_trainer.train()
model.save_lora("sft_saved_lora_3B")

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 580 | Num Epochs = 3 | Total steps = 108
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 16
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 16 x 1) = 16
 "-____-"     Trainable parameters = 12,156,928/3,000,000,000 (0.41% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
10,12.407400
20,11.445400
30,9.916600
40,8.660100
50,7.700900
60,6.780300
70,5.967000
80,5.321000
90,4.795200
100,4.403900


In [13]:
max_seq_length = 1024
lora_rank = 8

In [14]:
# ——————————————————————————————
# 4) GRPO stage, initialized from SFT adapter
# ——————————————————————————————

# re‑load the same base & adapter structure
# model, tokenizer = FastLanguageModel.from_pretrained(
#     model_name             = "unsloth/Llama-3.2-3B-Instruct",
#     max_seq_length         = max_seq_length,
#     load_in_4bit           = True,
#     fast_inference         = True,
#     max_lora_rank          = lora_rank,
#     gpu_memory_utilization = 0.9,
# )
# model = FastLanguageModel.get_peft_model(
#     model          = model,
#     r              = lora_rank,
#     target_modules = ["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"],
#     lora_alpha     = lora_rank,
# )
# load your SFT‐trained adapter
model.load_lora("sft_saved_lora_3B")

LoRARequest(lora_name='0', lora_int_id=0, lora_path='sft_saved_lora_3B', lora_tensors=None, lora_config=(None,), lora_local_path=None, long_lora_max_len=None, base_model_name=None, lora_embeddings=None)

In [15]:
def thinking_reward_func(completions, **kwargs) -> list[float]:
    """Reward function for including thinking tags"""
    rewards = []
    for completion in completions:
        try:
            reward = 0.0
            # Extract all thinking blocks
            for message in completion:
                if message["role"] == "assistant" and message.get("content"):
                    content = message["content"]

                    # Count opening and closing tags
                    opening_tags = len(re.findall(r"<thinking>", content))
                    closing_tags = len(re.findall(r"</thinking>", content))

                    if opening_tags == 0 or closing_tags == 0:
                        continue

                    if opening_tags == closing_tags:
                        reward += 0.5
                    else:
                        reward += 0.1
            reward = min(reward, 1.5)
            rewards.append(reward)
        except Exception as e:
            print(f"{RED}Error in thinking_reward_func: {e}{RESET}")
            rewards.append(0.0)
    assert len(rewards) == len(completions)
    return rewards


def answer_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function for including answer tags"""
    rewards = []
    for completion in completions:
        try:
            reward = 0.0
            # Extract all answer blocks
            for message in completion:
                if message["role"] == "assistant" and message.get("content"):
                    content = message["content"]

                    # Count opening and closing tags
                    opening_tags = len(re.findall(r"<answer>", content))
                    closing_tags = len(re.findall(r"</answer>", content))

                    if opening_tags == 0 or closing_tags == 0:
                        continue

                    if opening_tags == 1 and closing_tags == 1:
                        reward += 0.5
                    else:
                        reward += 0.1
            reward = min(reward, 1.5)
            rewards.append(reward)
        except Exception as e:
            print(f"{RED}Error in thinking_reward_func: {e}{RESET}")
            rewards.append(0.0)
    assert len(rewards) == len(completions)
    return rewards


def correctness_reward_func(prompts, completions, answer, **kwargs) -> list[float]:
    """Safe, robust reward for NYT Connections."""
    rewards = []
    
    for completion, expected_answer in zip(completions, answer):
        # default
        reward = 0.0

        # 1) Extract last <answer>…</answer> block
        predicted_block = None
        for msg in completion:
            if msg.get("role") == "assistant" and msg.get("content"):
                matches = re.findall(r"<answer>(.*?)</answer>", msg["content"], re.DOTALL)
                if matches:
                    predicted_block = matches[-1].strip()
        if not predicted_block:
            rewards.append(0.0)
            continue

        # 2) Safe parse
        try:
            pred_groups = ast.literal_eval(predicted_block)
            exp_groups  = ast.literal_eval(expected_answer)
        except Exception:
            rewards.append(0.0)
            continue

        # 3) Normalize: keep only lists of strings
        def clean(groups):
            cleaned = []
            for g in groups:
                if isinstance(g, list):
                    cleaned.append([w for w in g if isinstance(w, str)])
            return cleaned

        pred = clean(pred_groups)
        exp  = clean(exp_groups)

        # 4) Scoring
        # Perfect match
        if pred == exp:
            rewards.append(6.0)
            continue

        # +1.5 for each fully correct group (set‐equality)
        used = set()
        for pg in pred:
            for i, eg in enumerate(exp):
                if i in used: 
                    continue
                if set(pg) == set(eg):
                    reward += 1.5
                    used.add(i)
                    break

        # +0.75 for any 3‐word overlap in unmatched groups
        for pg in pred:
            for i, eg in enumerate(exp):
                if i in used:
                    continue
                overlap = len([w for w in pg if w in eg])
                if overlap == 3:
                    reward += 0.75
                    used.add(i)
                    break

        # +0.25 if every predicted group has exactly 4 words
        if all(len(pg) == 4 for pg in pred):
            reward += 0.25

        # +0.5 if no word is repeated across all predicted groups
        flat = [w for pg in pred for w in pg]
        if len(flat) == len(set(flat)):
            reward += 0.5

        rewards.append(reward)

    assert len(rewards) == len(completions)
    return rewards

In [16]:
from trl import GRPOConfig, GRPOTrainer

rl_args = GRPOConfig(
    use_vllm                     = False,
    learning_rate                = 5e-6,
    adam_beta1                   = 0.9,
    adam_beta2                   = 0.99,
    weight_decay                 = 0.1,
    warmup_ratio                 = 0.1,
    lr_scheduler_type            = "cosine",
    optim                        = "adamw_8bit",
    logging_steps                = 1,
    bf16                         = is_bfloat16_supported(),
    fp16                         = not is_bfloat16_supported(),
    per_device_train_batch_size  = 1,
    gradient_accumulation_steps  = 16,
    num_generations              = 4,
    max_prompt_length            = 128,
    max_completion_length        = 64,
    num_train_epochs             = 1,
    max_steps                    = 20,
    save_steps                   = 10,
    max_grad_norm                = 0.1,
    report_to                    = "none",
    output_dir                   = "outputs_grpo",
)

trainer = GRPOTrainer(
    model            = model,
    processing_class = tokenizer,
    reward_funcs     = [
        thinking_reward_func,
        correctness_reward_func,
        answer_format_reward_func,
    ],
    args             = rl_args,
    train_dataset    = train_ds,  # or a mix of train & test
)

Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Unsloth: We now expect `per_device_train_batch_size` to be a multiple of `num_generations`.
We will change the batch size of 1 to the `num_generations` of 4


In [17]:
trainer.train()
model.save_lora("grpo_saved_lora_3B")

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 580 | Num Epochs = 1 | Total steps = 20
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 16
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 16 x 1) = 64
 "-____-"     Trainable parameters = 12,156,928/3,000,000,000 (0.41% trained)


Step,Training Loss,reward,reward_std,completion_length,kl,rewards / thinking_reward_func,rewards / correctness_reward_func,rewards / answer_format_reward_func
1,0.000100,0.000000,0.000000,63.546875,0.003452,0.000000,0.000000,0.000000
2,0.000100,0.000000,0.000000,62.062500,0.003735,0.000000,0.000000,0.000000
3,0.000100,0.000000,0.000000,63.265625,0.003245,0.000000,0.000000,0.000000
4,0.000100,0.000000,0.000000,63.937500,0.002942,0.000000,0.000000,0.000000
5,0.000100,0.000000,0.000000,62.531250,0.002635,0.000000,0.000000,0.000000
6,0.000100,0.000000,0.000000,62.734375,0.002539,0.000000,0.000000,0.000000
7,0.000100,0.000000,0.000000,61.421875,0.002405,0.000000,0.000000,0.000000
8,0.000100,0.000000,0.000000,63.187500,0.002030,0.000000,0.000000,0.000000
9,0.000100,0.000000,0.000000,63.171875,0.001857,0.000000,0.000000,0.000000
10,0.000100,0.000000,0.000000,62.312500,0.001967,0.000000,0.000000,0.000000


In [18]:
text = tokenizer.apply_chat_template([
    {"role" : "system", "content" : SYSTEM_PROMPT},
    {"role" : "user", "content" : "[FLY, SINK, SHOWER, SALSA, TAP, RACE, DIP, MODERN, FALL, SWING, CARROT, TEAR, DROP, TALK, BLAZE, BOOM]"},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    text,
    sampling_params = sampling_params,
    lora_request = model.load_lora("grpo_saved_lora_3B"),
)[0].outputs[0].text

output

Processed prompts: 100%|██████████| 1/1 [00:12<00:00, 12.74s/it, est. speed input: 36.10 toks/s, output: 80.37 toks/s]


"After examining the given list of words, I've categorized them into 4 groups with a shared theme. Here is my solution:\n\n[['FALL', 'SINK', 'SWING', 'DROP'],\n ['FLY', 'BOOM', 'TEAR', 'TALK'],\n ['MODERN', 'RACE', 'SALSA', 'DIP'],\n ['CARROT', 'SHOWER', 'BLAZE', 'TAP']]\n\nExplanation:\n- FALLING OBJECTS: 'FALL', 'SINK', 'SWING', 'DROP'\n- EXPLODING SOUNDS: 'BOOM', 'TEAR', 'BLAZE', 'TAP'\n- FOOD AND DIP: 'MODERN', 'RACE', 'SALSA', 'DIP'\n- VEGETABLES: 'CARROT', 'SHOWER', 'FALL' is incorrect, instead 'FALL' doesn't belong, 'CARROT' belongs in 'VEGETABLES', 'SHOWER' belongs in 'FALL' is incorrect, 'SHOWER' belongs in 'EXPLODING SOUNDS'  'SHOWER' is not an exploding sound, instead 'SHOWER' doesn't belong, 'SHOWER' belongs to 'FALL' is incorrect, 'SHOWER' is not a falling object, 'SHOWER' is a verb,  'SHOWER' belongs in 'VEGETABLES' is incorrect, 'SHOWER' doesn't belong, 'SHOWER' belongs in 'EXPLODING SOUNDS' no, 'SHOWER' is not a falling, exploding, or a dip, 'SHOWER' is a verb,  'SHOWER

[['FALL', 'SINK', 'SWING', 'DROP'],\n ['FLY', 'BOOM', 'TEAR', 'TALK'],\n ['MODERN', 'RACE', 'SALSA', 'DIP'],\n ['CARROT', 'SHOWER', 'BLAZE', 'TAP']]

[['DIP', 'DROP', 'FALL', 'SINK'], ['BLAZE', 'FLY', 'RACE', 'TEAR'], ['MODERN', 'SALSA', 'SWING', 'TAP'], ['BOOM', 'CARROT', 'SHOWER', 'TALK']]